### <center>2018 Winter CS101.06</center>

# <center>栈和队列</center>

##### <center>by tanzhuxiaqiu@huawei.com</center>

## 今日议程

1. 栈
2. 队列
3. 双端队列

## 栈

遵循后进先出(LIFO)规则的对象集合。

常见的实例：
- 虚拟内存的栈空间
- 浏览器的历史访问
- 编辑器的撤销

### 抽象数据类型（ADT）

如果用S表示一个栈的实例，S必须支持两个基本操作：

- S.push(e): 将一个元素e压入栈S的栈顶。
- S.pop()：如果栈S中至少含有一个元素，从栈S的栈顶弹出一个元素并返回其值；如果栈S为空，则抛出异常。

此外，为了方便操作，栈S还应支持：

- S.top()：返回栈顶的元素值，但并不弹出该元素；如果栈S为空，则抛出异常。
- S.is_empty(): 判断栈S是否为空，如果为空就返回True；反之返回False。
- len(S): 返回栈S中元素的个数。

### 基于Python的List实现栈

|栈方法|对应List实现|
|---|---|
|S.push(e)|L.append(e)|
|S.pop()|L.pop()|
|S.top()|L[-1]|
|S.is_empty|len(L) == 0|
|len(S)|len(L)|

In [1]:
class Empty(Exception):
    """Error attempting to access an element from an empty container.
    """
    pass


class ArrayStack:
    """LIFO Stack implementation using a Python list as underlying storage.
    """

    def __init__(self):
        """Create an empty stack."""
        self._data = []

    def __len__(self):
        """Return the # of elements in the stack."""
        return len(self._data)

    def is_empty(self):
        """Return Ture if the stack is empty."""
        return len(self._data) == 0

    def push(self, e):
        """Add element e to the top of stack."""
        self._data.append(e)

    def pop(self):
        """Remove and return the element from the top of the stack.
        Raise Empty exception if the stack is empty.
        """
        if self.is_empty():
            raise Empty("Stack is empty")
        return self._data.pop()

    def top(self):
        """Return (not remove) the element at the top of the stack.
        Raise Empty exception if the stack is empty.
        """
        if self.is_empty():
            raise Empty("Stack is empty")
        return self._data[-1]

In [11]:
S = ArrayStack()
S.is_empty()

True

In [12]:
S.push(3)
S.push('a')
S.push({'k1': 'v1', 'k2': 'v2'})
S.pop()

{'k1': 'v1', 'k2': 'v2'}

In [13]:
len(S)

2

In [14]:
S.top()

'a'

In [15]:
S._data

[3, 'a']

# Any Questions?